In [ ]:
!pip install digits_to_word_nepali

TEST

In [ ]:
from digits_to_word_nepali import digit_to_nepali_words
result = digit_to_nepali_words(3748)
print(result)


तीन हजार सात सय अठचालीस


CLEANING FROM HERE

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

tsv_path = '/content/drive/MyDrive/TTSMajorProject/Notebooks/data/openslr/SLR43/ne_np_female/line_index.tsv'


df = pd.read_csv(tsv_path, sep='\t', header=None, names=['filename', 'text'])
df.head()

,filename,text
0,nep_0258_0119737288,दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्...
1,nep_0258_0461984530,डिग्रा देवीको जन्म सुदूरपश्चिम नेपालको बझाङ जि...
2,nep_0258_0576399736,टेकबहादुर ऐरको जन्म सुदूरपश्चिम नेपालको डडेलधु...
3,nep_0258_0707220145,सुमन शेखरमानन्धर नेपालको कृषि अर्थविद् तथा गाय...
4,nep_0258_0838627393,आधिकारिक रूपमा पहिलो नेपाली चलचित्र नायक शिव श...


In [ ]:
import re
import unicodedata
from digits_to_word_nepali import digit_to_nepali_words

In [ ]:
def normalize_unicode(text):
    return unicodedata.normalize('NFC', text)

In [ ]:
def clean_symbols(text):
    # Keep Devanagari, numbers, basic punctuation
    text = re.sub(r'[^ऀ-ॿ0-9.,?! ]+', '', text)
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
def split_long_text(text, max_words=15):
    # Split by punctuation first
    sentences = re.split(r'[।.!?]', text)
    chunks = []
    for sent in sentences:
        words = sent.strip().split()
        for i in range(0, len(words), max_words):
            chunk = ' '.join(words[i:i+max_words])
            if chunk:
                chunks.append(chunk)
    return chunks

In [ ]:
def expand_numbers(text):
    matches = re.findall(r'[०-९0-9]+', text)
    for m in matches:
        try:
            # Convert Devanagari digits to Arabic numerals for conversion)
            dev_to_int = {'०':'0','१':'1','२':'2','३':'3','४':'4','५':'5','६':'6','७':'7','८':'8','९':'9'}
            arabic_num = ''.join([dev_to_int.get(c, c) for c in m])
            # Convert to Nepali words using digit_to_nepali_words
            text = text.replace(m, digit_to_nepali_words(int(arabic_num)))
        except:
            continue
    return text

In [ ]:
STOP_TOKEN = "[STOP]"

def add_stop_token(text):
    return text + " " + STOP_TOKEN

In [ ]:
def preprocess_text_pipeline(text):
    text = normalize_unicode(text)      # Unicode normalization
    text = expand_numbers(text)         # Convert numbers to Nepali words
    text = clean_symbols(text)          # Remove unwanted symbols
    chunks = split_long_text(text)      # Split long sentences
    chunks = [add_stop_token(c) for c in chunks]  # Add stop toke
    return chunks

In [ ]:
# Apply preprocessing
df['chunks'] = df['text'].apply(preprocess_text_pipeline)
df_exploded = df.explode('chunks').reset_index(drop=True)
df_exploded = df_exploded.rename(columns={'chunks':'text_clean'})
output_path = '/content/drive/MyDrive/TTSMajorProject/Notebooks/data/clean_data/line_index_clean.tsv'
df_exploded.to_csv(output_path, sep='\t', index=False)
print("Check:", output_path)

Check: /content/drive/MyDrive/TTSMajorProject/Notebooks/data/clean_data/line_index_clean.tsv


In [ ]:
df_exploded.head(10)


,filename,text,text_clean
0,nep_0258_0119737288,दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्...,दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्...
1,nep_0258_0461984530,डिग्रा देवीको जन्म सुदूरपश्चिम नेपालको बझाङ जि...,डिग्रा देवीको जन्म सुदूरपश्चिम नेपालको बझाङ जि...
2,nep_0258_0576399736,टेकबहादुर ऐरको जन्म सुदूरपश्चिम नेपालको डडेलधु...,टेकबहादुर ऐरको जन्म सुदूरपश्चिम नेपालको डडेलधु...
3,nep_0258_0707220145,सुमन शेखरमानन्धर नेपालको कृषि अर्थविद् तथा गाय...,सुमन शेखरमानन्धर नेपालको कृषि अर्थविद् तथा गाय...
4,nep_0258_0838627393,आधिकारिक रूपमा पहिलो नेपाली चलचित्र नायक शिव श...,आधिकारिक रूपमा पहिलो नेपाली चलचित्र नायक शिव श...
5,nep_0258_1082186644,नेपाली सङ्गीतमा पुर्‍याएको योगदानका लागि सङ्गी...,नेपाली सङ्गीतमा पुर्याएको योगदानका लागि सङ्गीत...
6,nep_0258_1139532706,रेडियो नेपालमा कार्यरत रहँदा मास्टर रत्नदास प्...,रेडियो नेपालमा कार्यरत रहँदा मास्टर रत्नदास प्...
7,nep_0258_2044831682,प्रकाश श्रेष्ठ आधुनिक नेपाली गायक हुन्,प्रकाश श्रेष्ठ आधुनिक नेपाली गायक हुन् [STOP]
8,nep_0258_2658340102,सेते नेपाली श्रोतामाझ भिजेका गायक हुन्,सेते नेपाली श्रोतामाझ भिजेका गायक हुन् [STOP]
9,nep_0258_2670176669,शान्ति धामी एक लोकप्रिय लोक गायक हुन्,शान्ति धामी एक लोकप्रिय लोक गायक हुन् [STOP]


CHECK OF NUMBER TO TEXT MAP


In [ ]:
dev_digit_rows = df[df['text'].str.contains(r'[०-९]', regex=True)]
dev_digit_rows.head(10)

,filename,text,chunks
1473,nep_3997_3404972096,सीता बुढानिलकण्ठ स्कुलमा कक्षा दसमा र छोरा सज्...,[सीता बुढानिलकण्ठ स्कुलमा कक्षा दसमा र छोरा सज...


In [74]:
#end